# Segmenting and Clustering Neighborhoods in Toronto


## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

   ### Importing necessery packages

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize as json_to_df
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

# Segmenting and Clustering Neighborhoods in Toronto

## 1. Data scraping

#### Description: 
The following Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M contains the data that is in the table of postal codes which are needed and thus we need to crawl the url, retrieve the data, parse it and then transform it into a pandas dataframe.


In [2]:
# Define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood'] 

# Create the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Postal Code,Borough,Neighborhood


#### Web crawling/scraping
##### Importing data from provided URL

In [3]:
#Postal code URL
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Retrieve the wikipedia page
page = urlopen(wiki_url)

#Parse HTML
soup = BeautifulSoup(page, "lxml")

#Get table 
postal_code_table = soup.find('table', class_='wikitable sortable')

#### Extract the fields from table and place them into the dataframe
Iterate through each row and remove all newlines and add the data to approriate list

In [4]:
#Field lists
postal_code = []
borough = []
neighborhood = []
table_data = postal_code_table.findAll("tr")[1::]


for row in table_data:
    c, b, n = row.findAll('td')
    """
    postal_code.append(c.find(text=True))
    borough.append(b.find(text=True))
    neighborhood.append(n.find(text=True))
    """
    postal_code.append(c.text.replace("\n", ""))
    borough.append(b.text.replace("\n", ""))
    neighborhood.append(n.text.replace("\n", ""))

        
#Adding Data to our DataFrame
neighborhoods['Postal Code'] = postal_code
neighborhoods['Borough'] = borough
neighborhoods['Neighborhood'] = neighborhood
neighborhoods

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
#Drop rows where the Borough is "Not Assigned"
neighborhoods = neighborhoods.drop(neighborhoods[neighborhoods['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
neighborhoods.index = pd.RangeIndex(len(neighborhoods.index))
neighborhoods

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### If a cell has a borough but the neighborhood is  Not assigned, the neighborhood will be the same as the borough.

In [6]:
neighborhoods_reassigned = neighborhoods

#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index, row in neighborhoods.iterrows():
    if neighborhoods.loc[row_index, ['Neighborhood']].values.astype('str') == 'Not assigned':
        neighborhoods_reassigned.loc[row_index, ['Neighborhood']] = neighborhoods_reassigned.loc[row_index, ['Borough']].values.astype('str') 
        
neighborhoods_reassigned               

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


##### More than one neighborhood can exist in one postal code area.

In [7]:
neighborhoods_postal = neighborhoods_reassigned.groupby('Postal Code').agg(
    {'Borough':'first', 'Neighborhood': ', '.join}).reset_index()

print(neighborhoods_postal.shape)
neighborhoods_postal

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Filtering Neighborhood
Creating a dataframe dropping the neighborhoods which don't belong to Toronto

In [8]:
neighborhoods_not_toronto = neighborhoods_postal.drop(neighborhoods_postal[neighborhoods_postal['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighborhoods_not_toronto.index = pd.RangeIndex(len(neighborhoods_not_toronto.index))
neighborhoods_not_toronto

,Postal Code,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M4L,East Toronto,"The Beaches West, India Bazaar"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Moore Park, Summerhill East"
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


In [9]:
#Using the ungrouped dataframe to add locations after dropping the neighborhoods that don't belong to Toronto
neighborhoods_reassigned = neighborhoods_reassigned.drop(neighborhoods_reassigned[neighborhoods_reassigned['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighborhoods_reassigned.index = pd.RangeIndex(len(neighborhoods_reassigned.index))

neighborhoods_reassigned

,Postal Code,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M5A,Downtown Toronto,Regent Park
2,M5B,Downtown Toronto,Ryerson
3,M5B,Downtown Toronto,Garden District
4,M5C,Downtown Toronto,St. James Town
5,M4E,East Toronto,The Beaches
6,M5E,Downtown Toronto,Berczy Park
7,M5G,Downtown Toronto,Central Bay Street
8,M6G,Downtown Toronto,Christie
9,M5H,Downtown Toronto,Adelaide


## Retreive coordinates from link for higher relaibaility

In [10]:
!wget -q -O 'coord_postal_code.csv' http://cocl.us/Geospatial_data

#### Merging both frames on the postal code field to add the latitude and longitude to processed data

In [11]:
coordinates = pd.read_csv("./coord_postal_code.csv")
neighborhoods_w_coord = pd.merge(neighborhoods_reassigned, coordinates, on='Postal Code', how='left')
neighborhoods_w_coord

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


# Neighborhood analysis

#### Number of Boroughs and Neighborhoods

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods_w_coord['Borough'].unique()),
        neighborhoods_w_coord.shape[0]
    )
)

#Dropping NANs
neighborhoods_w_coord.dropna(inplace =True)
#Reset Index
neighborhoods_w_coord.index = pd.RangeIndex(len(neighborhoods_w_coord.index))
neighborhoods_w_coord


The dataframe has 4 boroughs and 74 neighborhoods.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


#### Using geopy library to get the latitude and longitude values of Toronto City.

In [13]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_w_coord['Latitude'], neighborhoods_w_coord['Longitude'], neighborhoods_w_coord['Borough'], neighborhoods_w_coord['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'MGXFGFJIL0WOWMIEFBY22MZACR4Y0CIGIKP552AK21A4IFJY' # your Foursquare ID
CLIENT_SECRET = 'TIASMHFUXM24RDBN1TMPSQOCIGSCVL12YGB2302VJRXBVBUG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's details


In [16]:
neighborhoods_w_coord.loc[0]
neighborhood_latitude = np.float(neighborhoods_w_coord.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(neighborhoods_w_coord.loc[0,['Longitude']].values)

#### Now, let's get the top 25 venues that are in Harbourfront within a radius of 500 meters.
First, let's create the GET request URL. Naming the URL url.

In [17]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude)

####  Extracting the  category of the venue

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Retrieve data using Foursquare API

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c882b44dd5797766e1bcc3f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

### Cleaning the json and structuring it into a pandas dataframe.

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_to_df(venues) # flatten JSON

# Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,The Distillery Historic District,Historic Site,43.650244,-79.359323


#### How many venues were returned by Foursquare?

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


### Exploring all Neighborhoods-


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius) #, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Toronto Venues

In [23]:
toronto_venues = getNearbyVenues(names=neighborhoods_w_coord['Neighborhood'],
                                   latitudes=neighborhoods_w_coord['Latitude'],
                                   longitudes=neighborhoods_w_coord['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


#### Size of dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1634, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Bathurst Quay,14,14,14,14,14,14
Berczy Park,30,30,30,30,30,30
Brockton,20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,20,20,20,20,20,20
CN Tower,14,14,14,14,14,14
Cabbagetown,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30


#### Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 184 uniques categories.


## 3. Analyze Each Neighborhood

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
toronto_onehot.shape

(1634, 184)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.100000,0.000000,0.033333,0.

In [30]:
toronto_grouped.shape

(73, 184)

Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0        Steakhouse  0.10
1              Café  0.07
2             Hotel  0.07
3  Asian Restaurant  0.07
4  Greek Restaurant  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3   Harbor / Marina  0.07
4           Airport  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1                Café  0.07
2  Seafood Restaurant  0.07
3      Farmers Market  0.07
4              Bakery  0.07


----Brockton----
                    venue  freq
0             Coffee Shop  0.10
1          Breakfast Spot  0.10
2                    Café  0.10
3  Furniture / Home Store  0.05
4    Caribbean Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.10
1         Yoga Studio  0.05
2          Comic Shop  0.05
3         Pizza Place  0.05
4    Recording Studio  0.05


----CN Tower----

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 7 venues for each neighborhood.

In [33]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Adelaide,Steakhouse,Hotel,Asian Restaurant,Café,Seafood Restaurant,Speakeasy,Bar
1,Bathurst Quay,Airport Terminal,Airport Lounge,Airport Service,Plane,Sculpture Garden,Boutique,Harbor / Marina
2,Berczy Park,Cocktail Bar,Bakery,Café,Seafood Restaurant,Farmers Market,Bistro,Italian Restaurant
3,Brockton,Café,Breakfast Spot,Coffee Shop,Convenience Store,Falafel Restaurant,Bar,Stadium
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Butcher,Smoke Shop,Spa,Brewery,Restaurant
5,CN Tower,Airport Terminal,Airport Lounge,Airport Service,Plane,Sculpture Garden,Boutique,Harbor / Marina
6,Cabbagetown,Restaurant,Coffee Shop,Italian Restaurant,Café,Bank,Japanese Restaurant,Beer Store
7,Central Bay Street,Coffee Shop,Bubble Tea Shop,Italian Restaurant,Spa,Park,Sushi Restaurant,Chinese Restaurant
8,Chinatown,Café,Caribbean Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery,Arts & Crafts Store,Dumpling Restaurant
9,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Diner,Nightclub


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 4 clusters.

In [34]:
# Set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
# labels = kmeans.labels_[0:63] 
labels = kmeans.labels_
labels

array([1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 3, 0, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 0, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
toronto_merged = neighborhoods_w_coord
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(74, 5)
(74,)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Park,Café,Bakery,Pub,Mexican Restaurant,Breakfast Spot
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3,Coffee Shop,Park,Café,Bakery,Pub,Mexican Restaurant,Breakfast Spot
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Café,Clothing Store,Gastropub,Music Venue,Plaza,Hotel,Shopping Mall
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Café,Clothing Store,Gastropub,Music Venue,Plaza,Hotel,Shopping Mall
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Restaurant,Italian Restaurant,Coffee Shop,Gastropub,Café,Japanese Restaurant,BBQ Joint


Finally, let's visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
#### Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
20,East Toronto,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Spa,Pub,Pizza Place,Juice Bar
22,Downtown Toronto,0,Coffee Shop,Café,Deli / Bodega,Steakhouse,Museum,Concert Hall,Gastropub
35,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Coworking Space,Discount Store
48,West Toronto,0,Coffee Shop,Café,Italian Restaurant,Diner,Sushi Restaurant,Pizza Place,Burrito Place


#### Cluster 2

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Café,Bakery,Pub,Mexican Restaurant,Breakfast Spot
2,Downtown Toronto,1,Café,Clothing Store,Gastropub,Music Venue,Plaza,Hotel,Shopping Mall
3,Downtown Toronto,1,Café,Clothing Store,Gastropub,Music Venue,Plaza,Hotel,Shopping Mall
4,Downtown Toronto,1,Restaurant,Italian Restaurant,Coffee Shop,Gastropub,Café,Japanese Restaurant,BBQ Joint
6,Downtown Toronto,1,Cocktail Bar,Bakery,Café,Seafood Restaurant,Farmers Market,Bistro,Italian Restaurant
7,Downtown Toronto,1,Coffee Shop,Bubble Tea Shop,Italian Restaurant,Spa,Park,Sushi Restaurant,Chinese Restaurant
8,Downtown Toronto,1,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Diner,Nightclub
9,Downtown Toronto,1,Steakhouse,Hotel,Asian Restaurant,Café,Seafood Restaurant,Speakeasy,Bar
10,Downtown Toronto,1,Steakhouse,Hotel,Asian Restaurant,Café,Seafood Restaurant,Speakeasy,Bar
11,Downtown Toronto,1,Steakhouse,Hotel,Asian Restaurant,Café,Seafood Restaurant,Speakeasy,Bar


#### Cluster 3

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
49,Central Toronto,2,Restaurant,Playground,Tennis Court,Wine Shop,Discount Store,Diner,Dessert Shop


#### Cluster 4

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,Downtown Toronto,3,Coffee Shop,Park,Café,Bakery,Pub,Mexican Restaurant,Breakfast Spot
5,East Toronto,3,Coffee Shop,Health Food Store,Pub,Wine Shop,Coworking Space,Dog Run,Discount Store
28,Downtown Toronto,3,Café,Hotel,Restaurant,Coffee Shop,Deli / Bodega,Gastropub,Ice Cream Shop
31,Central Toronto,3,Park,Swim School,Bus Line,Wine Shop,Coworking Space,Dog Run,Discount Store
34,Central Toronto,3,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Coworking Space,Discount Store
42,West Toronto,3,Gift Shop,Breakfast Spot,Coffee Shop,Cuban Restaurant,Dog Run,Bookstore,Bar
53,Downtown Toronto,3,Café,Caribbean Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery,Arts & Crafts Store,Dumpling Restaurant
